In [1]:
import pandas as pd
import altair as alt

In [2]:
# files = !ls ../data/ | rg visits
files = ['skillshare_2022_visits_0.csv',
         'skillshare_2022_visits_10.csv',
         'skillshare_2022_visits_11.csv',
         'skillshare_2022_visits_12.csv',
         'skillshare_2022_visits_13.csv',
         'skillshare_2022_visits_14.csv',
         'skillshare_2022_visits_15.csv',
         'skillshare_2022_visits_16.csv',
         'skillshare_2022_visits_17.csv',
         'skillshare_2022_visits_18.csv',
         'skillshare_2022_visits_19.csv',
         'skillshare_2022_visits_1.csv',
         'skillshare_2022_visits_20.csv',
         'skillshare_2022_visits_21.csv',
         'skillshare_2022_visits_22.csv',
         'skillshare_2022_visits_23.csv',
         'skillshare_2022_visits_24.csv',
         'skillshare_2022_visits_2.csv',
         'skillshare_2022_visits_3.csv',
         'skillshare_2022_visits_4.csv',
         'skillshare_2022_visits_5.csv',
         'skillshare_2022_visits_6.csv',
         'skillshare_2022_visits_7.csv',
         'skillshare_2022_visits_8.csv',
         'skillshare_2022_visits_9.csv']
files

['skillshare_2022_visits_0.csv',
 'skillshare_2022_visits_10.csv',
 'skillshare_2022_visits_11.csv',
 'skillshare_2022_visits_12.csv',
 'skillshare_2022_visits_13.csv',
 'skillshare_2022_visits_14.csv',
 'skillshare_2022_visits_15.csv',
 'skillshare_2022_visits_16.csv',
 'skillshare_2022_visits_17.csv',
 'skillshare_2022_visits_18.csv',
 'skillshare_2022_visits_19.csv',
 'skillshare_2022_visits_1.csv',
 'skillshare_2022_visits_20.csv',
 'skillshare_2022_visits_21.csv',
 'skillshare_2022_visits_22.csv',
 'skillshare_2022_visits_23.csv',
 'skillshare_2022_visits_24.csv',
 'skillshare_2022_visits_2.csv',
 'skillshare_2022_visits_3.csv',
 'skillshare_2022_visits_4.csv',
 'skillshare_2022_visits_5.csv',
 'skillshare_2022_visits_6.csv',
 'skillshare_2022_visits_7.csv',
 'skillshare_2022_visits_8.csv',
 'skillshare_2022_visits_9.csv']

In [3]:
# Load all together
visits = pd.DataFrame()
for file in files:
    visits = pd.concat([visits, pd.read_csv("../data/" + file, index_col=0)])
    
visits.visit_date = pd.to_datetime(visits.visit_date)
visits.head()

,uid,visit_id,visit_date,visit_duration,platform,marketing_channel,landing_event_name,has_signup,has_checkout_view,has_initiate_checkout,has_premium_start,is_existing_premium
0,20748409,614b25cf3f30af394f0d53081e30dcb4,2022-03-26,135,web,Direct,Viewed Membership Checkout,False,True,False,False,False
1,11534245,65b2dd6d29e9454372570dd1b6f92170,2022-03-26,1502,web,Other,Viewed-Class,False,False,False,False,True
2,23737452,00182e156cbe8efff389b7dcc4aefbbd,2022-03-26,0,web,Direct,View Home,False,False,False,False,True
3,10484775,11487111de21e9b6f720b9284bbe3755,2022-03-26,103,android_app,Other,View Home,False,False,False,False,True
4,12524843,9c898b4190705fb733701c6c75797d92,2022-03-26,1,android_app,Other,Launched-App,False,False,False,False,False


In [4]:
visits.has_signup.value_counts()

False    8273446
True      470430
Name: has_signup, dtype: int64

In [5]:
rolling_signups = visits\
    .set_index("visit_date")\
    .has_signup\
    .resample('1d')\
    .mean()\
    .rolling(4)\
    .mean()

rolling_signups

visit_date
2022-01-01         NaN
2022-01-02         NaN
2022-01-03         NaN
2022-01-04    0.168326
2022-01-05    0.148574
                ...   
2022-06-20    0.002485
2022-06-21    0.002779
2022-06-22    0.003341
2022-06-23    0.003259
2022-06-24    0.003208
Freq: D, Name: has_signup, Length: 175, dtype: float64

In [6]:
alt.Chart(
    rolling_signups.reset_index(),
    title="4-d Moving average of signups in 2022")\
    .mark_line()\
    .encode(
        x=alt.X("visit_date"),
        y=alt.Y("has_signup"))\
    .properties(
        width=700,
        height=600)

alt.Chart(...)

In [7]:
by_platform = visits\
    .groupby(
        [
            "platform", 
             pd.Grouper(key="visit_date", freq="1d")
        ])\
    .sum()\
    .reset_index()

by_platform["has_signup_moving_avg"] = by_platform.has_signup\
    .rolling(5)\
    .mean()

by_platform

,platform,visit_date,uid,visit_duration,has_signup,has_checkout_view,has_initiate_checkout,has_premium_start,is_existing_premium,has_signup_moving_avg
0,android_alpha_app,2022-03-24,7679538,0,0,0,0,0,0,NaN
1,android_alpha_app,2022-05-09,2559846,0,0,0,0,0,0,NaN
2,android_app,2022-01-01,25742668402,1127821,104,356,277,179,882,NaN
3,android_app,2022-01-02,37729383995,1793731,118,403,318,193,1454,NaN
4,android_app,2022-01-03,46124279574,2177797,108,402,318,207,1909,66.0
...,...,...,...,...,...,...,...,...,...,...
1762,web,2022-06-20,485685422309,18370776,63,1034,193,134,21202,63.4
1763,web,2022-06-21,101656343973,70488703,17,206,34,23,4477,48.4
1764,web,2022-06-22,99103715617,71391804,25,181,42,24,4397,41.4
1765,web,2022-06-23,105949295433,63490342,14,152,36,18,3947,35.6


In [8]:
selection = alt.selection_single(on="mouseover", nearest=True)

alt.Chart(
    by_platform)\
    .mark_line()\
    .encode(
        x=alt.X("visit_date"),
        y=alt.Y("has_signup_moving_avg"),
        tooltip=["platform", "visit_date"],
        color="platform")\
    .properties(
        width=700,
        height=600)\
    .add_selection(selection)

alt.Chart(...)